####  Work report from April-May 2018 on work done on sparse GPs in Stan


In this case study, I will briefly explain some sparse GP models implemented in Stan and compare them to full GP model on a simple 1-D Regression dataset. The main idea is to see how they work out in Stan using inference done through HMC sampling. Snelson dataset has 200 datapoints.


Initially, to make a valid comparison, I try out these experiments with optimisation of the hyperparameters by maximising marginal likelihood. This is done in GPy

[GP Regression on Snelson dataset](./GPy-Snelson.ipynb)


I start off with explaining MCMC variationally sparse GPs explained nicely in [2]. It is one of the core 6 models in GPFlow called as "Sparse Markov Chain Monte Carlo", 
Lets formulate the model first, we have some input data $X = \{X_n\}_{n=1}^{N}$ and  and output on those input locations, $Y = \{y_n\}_{n=1}^{N}$. A latent function is asuumed drawn from a GP with a 0 mean and a covariance function $k(x,x')$ with hyper parameters $\theta$. The prior over the function is defined by a Gaussian Process, the consistency property of GP allows us to write it over the set of input points in the form of a multivariate Gaussian distribution, $p(F|X) = \mathrm{N}(F|X, \bm{\theta})$. We here use a Gaussian observation model, $p(\bm{Y}|F)$, the observations are independent of each other given the latent function values. In order to make a prediction(function evaluation) at a test point $x^*$, $f^*$  can be given as:

$p(f^*|y) = \int \int p(f^*|u,\theta) p(u,\theta|y) d\theta du$

If we instead use the variational inducing point framework, we will work with a variational posterior $q(u,\theta)$ in place of the original posterior. This will lead to variational marginalisation of $u$ and $\theta$. 

$q(f^*) = \int \int p(f^*|u,\theta) q(u,\theta) d\theta du$

Now, for selecting a variational distribution is done by first selecting a family of simple densities and then find the member which is closest to the real distribution in terms of KL divergence. This is akin to maximising the ELBO quanitity with respect to variational parameters. Since these parameters are not model parameters, they are safe from overfitting, and we are not overparameterising this problem.

The optimal variational distribution is given in Equation 6 of [2]. 

$\log\hat{q}(u, \theta) = \mathbb{E}_{p(f|u, \theta)}[\log p(y|f)] +\log p(u|\theta) +\log p(\theta) - \log C $

Using the whitening trick where $u = Rv$, this can be written as: 

$ \log\hat{q}(u, \theta) = \mathbb{E}_{p(f|u, \theta)}[\log p(y|f)] +\log p(u|\theta) +\log p(\theta) $

The conditional Gaussian is given as:

$p(f|u, \theta) = \mathbb{N}(K_{fu}K_{uu}^{-1}u, K_{ff}-Q_{ff})$

$Q_{ff} = K_{fu}K_{uu}^{-1}K_{uf}$

While we put a lower rank GP prior over $u$ 

$p(u, \theta) = \mathbb{N}(0, K_{uu})$

This can be interpreted as the expected log likelihood, in the limit when $N=M$ and $f=u$, we will get the original full data likelihood. Since the likelihood is factorised, the integral can be written as one dimensional quadratures. In case where the likelihood is Gaussian, we can derive an analytic form for the bound over the conditional density $p(Y|u)$ given in Equation 6. We can use this density to sample jointly from $q(u, \theta)$ and the integral then can be approximated by an unbiased Monte Carlo sum of the samples. We obtain a Gaussian density for each posterior sample of $u, \theta$, the resulting density is then a mixture of Gaussian densities which is not Gaussian itself but we can calculate the moments analytically.

$  L1 =  \prod_{i=1}^{N}(y_i|\mu_i, \beta^{-1}) \exp(-0.5\beta \tilde{k_{ii}})$



The relevant notebooks are given here, first for 7 inducing points and second for 20 inducing points on training set of 200 points, 

[Variational Sparse GP for Snelson data with 7 inducing points](./GP_Sparse_Var_Snelson_M_7.ipynb)

[Variational Sparse GP for Snelson data with 20 inducing points](./GP_Sparse_Var_Snelson_M_20.ipynb)

To compare these approaches, we also obtained the results with a full GP predictive model given in this notebook:

[Full GP for Snelson data with predictions](./GP_Full_pred.ipynb)

We can compare the full GP model and the sparse approximations values on the test set through RMSE, NMSE and MLPPD(Mean log predictive posterior density).

We also show here a notebook on sparse GPs with random Fourier features. 

[Sparse Spectrum GP with RFF features sampled from frequency specturm of RBF kernel.](./GP_RBF_RFF.ipynb)


##### References:

[1] GPy GPy authors 2014 

[2] MCMC for Variationally Sparse Gaussian Processes J Hensman, A G de G Matthews, M Filippone, Z Ghahramani Advances in Neural Information Processing Systems, 1639-1647, 2015.

[3] The variational Gaussian approximation revisited M Opper, C Archambeau Neural computation 21 (3), 786-792, 2009.

[4] Scalable Variational Gaussian Process Classification J Hensman, A G de G Matthews, Z Ghahramani Proceedings of AISTATS 18, 2015.